GitHub Repo: https://github.com/jesmeen-online/SleepCon-Sleep-Pattern-and-Posture-Recognition.git

Research Paper: https://www.ijournalse.org/index.php/ESJ/article/view/961

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

import matplotlib.pyplot as plt



def build_sleep_posture_model(num_postures):
    model = Sequential()

    # First convolutional block
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)))  # Reduced number of filters
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # Second convolutional block
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # Third convolutional block
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    # Fourth convolutional block
#     model.add(Conv2D(256, (3, 3), activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Conv2D(256, (3, 3), activation='relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.4))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_postures, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model




def create_simple_model(num_postures):
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape = (256,256,1)))
    model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu'))
    
    model.add(MaxPooling2D(pool_size = 2))
    model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
    
    model.add(MaxPooling2D(pool_size = 2))
    model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu'))
    model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu'))
    
    model.add(MaxPooling2D(pool_size = 2))
    model.add(Flatten())
    model.add(Dropout(rate = 0.5))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(units = num_postures, activation = 'softmax'))
    model.compile(optimizer = "adam", loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model


In [2]:
# Directories for the dataset
train_dir = "src/data/train"
val_dir = "src/data/validation"
test_dir = "src/data/test"
# Data generators for loading and augmenting images
train_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

# validation_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_data_gen = ImageDataGenerator(rescale=1./255)

train_gen = train_data_gen.flow_from_directory(
    train_dir,
    target_size=(128, 128),  # Reduced input size
    color_mode='grayscale',
    batch_size=16,  # Reduced batch size
    class_mode='categorical'
)

validation_gen = validation_data_gen.flow_from_directory(
    val_dir,
    target_size=(128, 128),  # Reduced input size
    color_mode='grayscale',
    batch_size=16,  # Reduced batch size
    class_mode='categorical'
)

test_gen = test_data_gen.flow_from_directory(
    test_dir,
    target_size=(128, 128),  # Reduced input size
    color_mode='grayscale',
    batch_size=16,  # Reduced batch size
    class_mode='categorical',
    shuffle=False  # Important for evaluation
)

Found 4999 images belonging to 3 classes.
Found 1395 images belonging to 3 classes.
Found 1199 images belonging to 3 classes.


In [ ]:
# Create the model
sleep_posture_model = build_sleep_posture_model(num_postures=train_gen.num_classes)
sleep_posture_model.summary()

# Define callbacks
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=0.00001, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1)

# Train the model
training_history = sleep_posture_model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // train_gen.batch_size,
    validation_data=validation_gen,
    validation_steps=validation_gen.samples // validation_gen.batch_size,
    epochs=200,  # Reduced epochs for demonstration purposes
#     callbacks=[lr_reduction, early_stopping],
    verbose=1  # Set verbosity to 1 to show progress bar without detailed validation metrics
)

# Evaluate the model on the test set
test_loss, test_accuracy = sleep_posture_model.evaluate(test_gen, steps=test_gen.samples // test_gen.batch_size)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 124, 124, 32)      9248      
                                                                 
 batch_normalization_1 (Batc  (None, 124, 124, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 62, 32)       0         
 )                                                               
                                                        

Epoch 29/200
312/312 [==============================] - 41s 129ms/step - loss: 0.0883 - accuracy: 0.9693 - val_loss: 0.9140 - val_accuracy: 0.7378
Epoch 30/200
312/312 [==============================] - 42s 134ms/step - loss: 0.0697 - accuracy: 0.9761 - val_loss: 1.0539 - val_accuracy: 0.7234
Epoch 31/200
312/312 [==============================] - 39s 126ms/step - loss: 0.0744 - accuracy: 0.9703 - val_loss: 1.1910 - val_accuracy: 0.7076
Epoch 32/200
312/312 [==============================] - 40s 129ms/step - loss: 0.0667 - accuracy: 0.9781 - val_loss: 0.9874 - val_accuracy: 0.7845
Epoch 33/200
312/312 [==============================] - 43s 138ms/step - loss: 0.0639 - accuracy: 0.9775 - val_loss: 2.0632 - val_accuracy: 0.6509
Epoch 34/200
312/312 [==============================] - 40s 128ms/step - loss: 0.0707 - accuracy: 0.9775 - val_loss: 1.2669 - val_accuracy: 0.6595
Epoch 35/200
312/312 [==============================] - 40s 127ms/step - loss: 0.0717 - accuracy: 0.9765 - val_loss: 0

In [ ]:
# Plotting training acc/loss and val acc/loss
history=training_history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
range_ep = 200 +1
epoch_x = range(1, range_ep)

plt.plot(epoch_x,acc,'bo',label="Training Acc")
plt.plot(epoch_x,val_acc,'b',label='Validation Acc')
plt.title('Training and Validation Acc')
plt.legend()
plt.figure()

plt.plot(epoch_x,loss,'bo',label="Training Loss")
plt.plot(epoch_x,val_loss,'b',label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.figure()

plt.show()

In [ ]:
# Model weights and model
sleep_posture_model.save_weights('my_model_weights2.h5')
sleep_posture_model.save("my_model2.h5")